In [ ]:
# Model
!git clone "https://github.com/cybernetic-m/eai-project.git" # type: ignore

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim
from torch.optim.lr_scheduler import ExponentialLR 

# Others
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle

sys.path.append('/content/eai-project/training')
sys.path.append('/content/eai-project/preprocessing')
sys.path.append('/content/eai-project/dataset')
sys.path.append('/content/eai-project/utils')
sys.path.append('/content/eai-project/models')
sys.path.append('/content/eai-project/modules')
sys.path.append('/content/eai-project/testing')
sys.path.append('/content/eai-project')
from train import train
from preprocessing import *
from thermal_dataset import thermal_dataset 
from csv_utils import *
from complete_model import complete_model 
from testing.test import test
from blocks import mlp, linear, rnn, lstm
prefix = '/content'
    
pd.set_option('display.max_columns', None)

In [ ]:
# Set a seed for reproducibility purposes
seed = 46
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

# Set the device (cuda for Nvidia GPUs, mps for M1, M2 .. Apple Silicon)
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"